In [1]:
import pandas as pd
from datetime import datetime
from google.cloud import bigquery

### Ind_registrados

In [2]:
client = bigquery.Client()
query = """
With

registrado as (
  SELECT 
  platform_id,
  federation_id,
  email,
  cast(date(date_created)as string) as date_created,
  case when date_created < '2022-03-28'
  then cast(date_diff('2022-09-30', '2022-03-28', day) as string)
  else cast(date_diff('2022-09-30', date(date_created), day) as string)
  end as dias_registrado,
  case
  when
  date_diff('2022-09-30', date(date_created), day) < 30
  then 'menos de 30 días creado'
  else  'más de 30 días creado'
  end as condicion
  from `webhooks.usuarios_unicos` as users
  where users.date_created < '2022-09-30')

  Select * from registrado
"""

ind_registrados = client.query(query).result().to_dataframe(create_bqstorage_client=False)  

### Ind_total

In [3]:
ind_total = pd.read_csv("C:/Users/alopez/Desktop/indicadores/20220930/Ind_total.csv")
ind_total.columns

Index(['platform_id', 'sum_total', 'q_total', 'prom_total', 'freq_total'], dtype='object')

In [4]:
output1 = pd.merge(ind_registrados, ind_total, on= 'platform_id', how= 'left')
output1.columns

Index(['platform_id', 'federation_id', 'email', 'date_created',
       'dias_registrado', 'condicion', 'sum_total', 'q_total', 'prom_total',
       'freq_total'],
      dtype='object')

### Ind_30h

In [5]:
ind_30h = pd.read_csv("C:/Users/alopez/Desktop/indicadores/20220930/Ind_30h.csv")
ind_30h.columns

Index(['platform_id', 'sum_30h', 'q_30h', 'prom_30h', 'freq_30h'], dtype='object')

In [6]:
output2 = pd.merge(output1, ind_30h, on= 'platform_id', how= 'left')
output2.columns

Index(['platform_id', 'federation_id', 'email', 'date_created',
       'dias_registrado', 'condicion', 'sum_total', 'q_total', 'prom_total',
       'freq_total', 'sum_30h', 'q_30h', 'prom_30h', 'freq_30h'],
      dtype='object')

### Ind_live_30h

In [7]:
ind_live = pd.read_csv("C:/Users/alopez/Desktop/indicadores/20220930/Ind_live_30h.csv")
del ind_live['content_type']
ind_live.columns

Index(['platform_id', 'sum_live', 'q_live', 'prom_live', 'freq_live'], dtype='object')

In [8]:
output3 = pd.merge(output2, ind_live, on= 'platform_id', how= 'left')
output3.columns

Index(['platform_id', 'federation_id', 'email', 'date_created',
       'dias_registrado', 'condicion', 'sum_total', 'q_total', 'prom_total',
       'freq_total', 'sum_30h', 'q_30h', 'prom_30h', 'freq_30h', 'sum_live',
       'q_live', 'prom_live', 'freq_live'],
      dtype='object')

### Ind_od_30h

In [9]:
ind_od = pd.read_csv("C:/Users/alopez/Desktop/indicadores/20220930/Ind_od_30h.csv")
ind_od.columns

Index(['platform_id', 'sum_od', 'q_od', 'prom_od', 'freq_od'], dtype='object')

In [10]:
output4 = pd.merge(output3, ind_od, on= 'platform_id', how= 'left')
output4.columns

Index(['platform_id', 'federation_id', 'email', 'date_created',
       'dias_registrado', 'condicion', 'sum_total', 'q_total', 'prom_total',
       'freq_total', 'sum_30h', 'q_30h', 'prom_30h', 'freq_30h', 'sum_live',
       'q_live', 'prom_live', 'freq_live', 'sum_od', 'q_od', 'prom_od',
       'freq_od'],
      dtype='object')

### Ind_device_30h

In [11]:
ind_device = pd.read_csv("C:/Users/alopez/Desktop/indicadores/20220930/Ind_device_30h.csv")
ind_device_desk = ind_device[ind_device['device_type']=='Desktop']
ind_device_desk.rename(columns={'sum_device':'sum_desk', 'q_device': 'q_desk', 
'prom_device': 'prom_desk', 'freq_device': 'freq_desk'},inplace = True)
del ind_device_desk['device_type']
ind_device_phone = ind_device[ind_device['device_type']=='Phone']
ind_device_phone.rename(columns={'sum_device':'sum_phone', 'q_device': 'q_phone', 
'prom_device': 'prom_phone', 'freq_device': 'freq_phone'},inplace = True)
del ind_device_phone['device_type']

c:\Users\alopez\Anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [12]:
output5 = pd.merge(output4, ind_device_desk, on= 'platform_id', how= 'left')
output5.columns

Index(['platform_id', 'federation_id', 'email', 'date_created',
       'dias_registrado', 'condicion', 'sum_total', 'q_total', 'prom_total',
       'freq_total', 'sum_30h', 'q_30h', 'prom_30h', 'freq_30h', 'sum_live',
       'q_live', 'prom_live', 'freq_live', 'sum_od', 'q_od', 'prom_od',
       'freq_od', 'sum_desk', 'q_desk', 'prom_desk', 'freq_desk'],
      dtype='object')

In [13]:
output5 = pd.merge(output5, ind_device_phone, on= 'platform_id', how= 'left')
output5.columns

Index(['platform_id', 'federation_id', 'email', 'date_created',
       'dias_registrado', 'condicion', 'sum_total', 'q_total', 'prom_total',
       'freq_total', 'sum_30h', 'q_30h', 'prom_30h', 'freq_30h', 'sum_live',
       'q_live', 'prom_live', 'freq_live', 'sum_od', 'q_od', 'prom_od',
       'freq_od', 'sum_desk', 'q_desk', 'prom_desk', 'freq_desk', 'sum_phone',
       'q_phone', 'prom_phone', 'freq_phone'],
      dtype='object')

### Ind_bloque_30h

In [14]:
ind_bloque = pd.read_csv("C:/Users/alopez/Desktop/indicadores/20220930/Ind_bloque_30h.csv")
ind_bloque.columns

Index(['platform_id', 'bloque', 'sum_bloque', 'q_bloque', 'prom_min',
       'freq_bloque'],
      dtype='object')

In [15]:
ind_bloque_6a7 = ind_bloque[ind_bloque['bloque'] == '6 a 7']
ind_bloque_6a7.rename(columns={'sum_bloque':'sum_6a7', 'q_bloque': 'q_6a7', 
'prom_min': 'prom_6a7', 'freq_bloque': 'freq_6a7'},inplace = True)
del ind_bloque_6a7['bloque']

ind_bloque_7a9 = ind_bloque[ind_bloque['bloque'] == '7 a 9']
ind_bloque_7a9.rename(columns={'sum_bloque':'sum_7a9', 'q_bloque': 'q_7a9', 
'prom_min': 'prom_7a9', 'freq_bloque': 'freq_7a9'},inplace = True)
del ind_bloque_7a9['bloque']

ind_bloque_9a13 = ind_bloque[ind_bloque['bloque'] == '9 a 13']
ind_bloque_9a13.rename(columns={'sum_bloque':'sum_9a13', 'q_bloque': 'q_9a13', 
'prom_min': 'prom_9a13', 'freq_bloque': 'freq_9a13'},inplace = True)
del ind_bloque_9a13['bloque']

ind_bloque_13a16 = ind_bloque[ind_bloque['bloque'] == '13 a 16']
ind_bloque_13a16.rename(columns={'sum_bloque':'sum_13a16', 'q_bloque': 'q_13a16', 
'prom_min': 'prom_13a16', 'freq_bloque': 'freq_13a16'},inplace = True)
del ind_bloque_13a16['bloque']

ind_bloque_16a18 = ind_bloque[ind_bloque['bloque'] == '16 a 18']
ind_bloque_16a18.rename(columns={'sum_bloque':'sum_16a18', 'q_bloque': 'q_16a18', 
'prom_min': 'prom_16a18', 'freq_bloque': 'freq_16a18'},inplace = True)
del ind_bloque_16a18['bloque']

ind_bloque_18a21 = ind_bloque[ind_bloque['bloque'] == '18 a 21']
ind_bloque_18a21.rename(columns={'sum_bloque':'sum_18a21', 'q_bloque': 'q_18a21', 
'prom_min': 'prom_18a21', 'freq_bloque': 'freq_18a21'},inplace = True)
del ind_bloque_18a21['bloque']

ind_bloque_21a06 = ind_bloque[ind_bloque['bloque'] == '21 a 06']
ind_bloque_21a06.rename(columns={'sum_bloque':'sum_21a06', 'q_bloque': 'q_21a06', 
'prom_min': 'prom_21a06', 'freq_bloque': 'freq_21a06'},inplace = True)
del ind_bloque_21a06['bloque']



In [16]:
output6 = pd.merge(output5, ind_bloque_6a7, on= 'platform_id', how= 'left')
output6.columns

Index(['platform_id', 'federation_id', 'email', 'date_created',
       'dias_registrado', 'condicion', 'sum_total', 'q_total', 'prom_total',
       'freq_total', 'sum_30h', 'q_30h', 'prom_30h', 'freq_30h', 'sum_live',
       'q_live', 'prom_live', 'freq_live', 'sum_od', 'q_od', 'prom_od',
       'freq_od', 'sum_desk', 'q_desk', 'prom_desk', 'freq_desk', 'sum_phone',
       'q_phone', 'prom_phone', 'freq_phone', 'sum_6a7', 'q_6a7', 'prom_6a7',
       'freq_6a7'],
      dtype='object')

In [17]:
output7 = pd.merge(output6, ind_bloque_7a9, on= 'platform_id', how= 'left')
output7.columns

Index(['platform_id', 'federation_id', 'email', 'date_created',
       'dias_registrado', 'condicion', 'sum_total', 'q_total', 'prom_total',
       'freq_total', 'sum_30h', 'q_30h', 'prom_30h', 'freq_30h', 'sum_live',
       'q_live', 'prom_live', 'freq_live', 'sum_od', 'q_od', 'prom_od',
       'freq_od', 'sum_desk', 'q_desk', 'prom_desk', 'freq_desk', 'sum_phone',
       'q_phone', 'prom_phone', 'freq_phone', 'sum_6a7', 'q_6a7', 'prom_6a7',
       'freq_6a7', 'sum_7a9', 'q_7a9', 'prom_7a9', 'freq_7a9'],
      dtype='object')

In [18]:
output8 = pd.merge(output7, ind_bloque_9a13, on= 'platform_id', how= 'left')
output8.columns

Index(['platform_id', 'federation_id', 'email', 'date_created',
       'dias_registrado', 'condicion', 'sum_total', 'q_total', 'prom_total',
       'freq_total', 'sum_30h', 'q_30h', 'prom_30h', 'freq_30h', 'sum_live',
       'q_live', 'prom_live', 'freq_live', 'sum_od', 'q_od', 'prom_od',
       'freq_od', 'sum_desk', 'q_desk', 'prom_desk', 'freq_desk', 'sum_phone',
       'q_phone', 'prom_phone', 'freq_phone', 'sum_6a7', 'q_6a7', 'prom_6a7',
       'freq_6a7', 'sum_7a9', 'q_7a9', 'prom_7a9', 'freq_7a9', 'sum_9a13',
       'q_9a13', 'prom_9a13', 'freq_9a13'],
      dtype='object')

In [19]:
output9 = pd.merge(output8, ind_bloque_13a16, on= 'platform_id', how= 'left')
output9.columns

Index(['platform_id', 'federation_id', 'email', 'date_created',
       'dias_registrado', 'condicion', 'sum_total', 'q_total', 'prom_total',
       'freq_total', 'sum_30h', 'q_30h', 'prom_30h', 'freq_30h', 'sum_live',
       'q_live', 'prom_live', 'freq_live', 'sum_od', 'q_od', 'prom_od',
       'freq_od', 'sum_desk', 'q_desk', 'prom_desk', 'freq_desk', 'sum_phone',
       'q_phone', 'prom_phone', 'freq_phone', 'sum_6a7', 'q_6a7', 'prom_6a7',
       'freq_6a7', 'sum_7a9', 'q_7a9', 'prom_7a9', 'freq_7a9', 'sum_9a13',
       'q_9a13', 'prom_9a13', 'freq_9a13', 'sum_13a16', 'q_13a16',
       'prom_13a16', 'freq_13a16'],
      dtype='object')

In [20]:
output10 = pd.merge(output9, ind_bloque_16a18, on= 'platform_id', how= 'left')
output10.columns

Index(['platform_id', 'federation_id', 'email', 'date_created',
       'dias_registrado', 'condicion', 'sum_total', 'q_total', 'prom_total',
       'freq_total', 'sum_30h', 'q_30h', 'prom_30h', 'freq_30h', 'sum_live',
       'q_live', 'prom_live', 'freq_live', 'sum_od', 'q_od', 'prom_od',
       'freq_od', 'sum_desk', 'q_desk', 'prom_desk', 'freq_desk', 'sum_phone',
       'q_phone', 'prom_phone', 'freq_phone', 'sum_6a7', 'q_6a7', 'prom_6a7',
       'freq_6a7', 'sum_7a9', 'q_7a9', 'prom_7a9', 'freq_7a9', 'sum_9a13',
       'q_9a13', 'prom_9a13', 'freq_9a13', 'sum_13a16', 'q_13a16',
       'prom_13a16', 'freq_13a16', 'sum_16a18', 'q_16a18', 'prom_16a18',
       'freq_16a18'],
      dtype='object')

In [21]:
output11 = pd.merge(output10, ind_bloque_18a21, on= 'platform_id', how= 'left')
output11.columns

Index(['platform_id', 'federation_id', 'email', 'date_created',
       'dias_registrado', 'condicion', 'sum_total', 'q_total', 'prom_total',
       'freq_total', 'sum_30h', 'q_30h', 'prom_30h', 'freq_30h', 'sum_live',
       'q_live', 'prom_live', 'freq_live', 'sum_od', 'q_od', 'prom_od',
       'freq_od', 'sum_desk', 'q_desk', 'prom_desk', 'freq_desk', 'sum_phone',
       'q_phone', 'prom_phone', 'freq_phone', 'sum_6a7', 'q_6a7', 'prom_6a7',
       'freq_6a7', 'sum_7a9', 'q_7a9', 'prom_7a9', 'freq_7a9', 'sum_9a13',
       'q_9a13', 'prom_9a13', 'freq_9a13', 'sum_13a16', 'q_13a16',
       'prom_13a16', 'freq_13a16', 'sum_16a18', 'q_16a18', 'prom_16a18',
       'freq_16a18', 'sum_18a21', 'q_18a21', 'prom_18a21', 'freq_18a21'],
      dtype='object')

In [22]:
output12 = pd.merge(output11, ind_bloque_21a06, on= 'platform_id', how= 'left')
output12.columns

Index(['platform_id', 'federation_id', 'email', 'date_created',
       'dias_registrado', 'condicion', 'sum_total', 'q_total', 'prom_total',
       'freq_total', 'sum_30h', 'q_30h', 'prom_30h', 'freq_30h', 'sum_live',
       'q_live', 'prom_live', 'freq_live', 'sum_od', 'q_od', 'prom_od',
       'freq_od', 'sum_desk', 'q_desk', 'prom_desk', 'freq_desk', 'sum_phone',
       'q_phone', 'prom_phone', 'freq_phone', 'sum_6a7', 'q_6a7', 'prom_6a7',
       'freq_6a7', 'sum_7a9', 'q_7a9', 'prom_7a9', 'freq_7a9', 'sum_9a13',
       'q_9a13', 'prom_9a13', 'freq_9a13', 'sum_13a16', 'q_13a16',
       'prom_13a16', 'freq_13a16', 'sum_16a18', 'q_16a18', 'prom_16a18',
       'freq_16a18', 'sum_18a21', 'q_18a21', 'prom_18a21', 'freq_18a21',
       'sum_21a06', 'q_21a06', 'prom_21a06', 'freq_21a06'],
      dtype='object')

### Ind_30ih

In [23]:
ind_30ih = pd.read_csv("C:/Users/alopez/Desktop/indicadores/20220930/Ind_30ih.csv")
ind_30ih.columns

Index(['platform_id', 'sum_30ih', 'q_30ih', 'prom_30ih', 'freq_30ih'], dtype='object')

In [24]:
output13 = pd.merge(output12, ind_30ih, on= 'platform_id', how= 'left')
output13.columns

Index(['platform_id', 'federation_id', 'email', 'date_created',
       'dias_registrado', 'condicion', 'sum_total', 'q_total', 'prom_total',
       'freq_total', 'sum_30h', 'q_30h', 'prom_30h', 'freq_30h', 'sum_live',
       'q_live', 'prom_live', 'freq_live', 'sum_od', 'q_od', 'prom_od',
       'freq_od', 'sum_desk', 'q_desk', 'prom_desk', 'freq_desk', 'sum_phone',
       'q_phone', 'prom_phone', 'freq_phone', 'sum_6a7', 'q_6a7', 'prom_6a7',
       'freq_6a7', 'sum_7a9', 'q_7a9', 'prom_7a9', 'freq_7a9', 'sum_9a13',
       'q_9a13', 'prom_9a13', 'freq_9a13', 'sum_13a16', 'q_13a16',
       'prom_13a16', 'freq_13a16', 'sum_16a18', 'q_16a18', 'prom_16a18',
       'freq_16a18', 'sum_18a21', 'q_18a21', 'prom_18a21', 'freq_18a21',
       'sum_21a06', 'q_21a06', 'prom_21a06', 'freq_21a06', 'sum_30ih',
       'q_30ih', 'prom_30ih', 'freq_30ih'],
      dtype='object')

In [25]:
output13["fecha"] =  datetime(2022,9,30).strftime('%Y-%m-%d')
output13.columns

Index(['platform_id', 'federation_id', 'email', 'date_created',
       'dias_registrado', 'condicion', 'sum_total', 'q_total', 'prom_total',
       'freq_total', 'sum_30h', 'q_30h', 'prom_30h', 'freq_30h', 'sum_live',
       'q_live', 'prom_live', 'freq_live', 'sum_od', 'q_od', 'prom_od',
       'freq_od', 'sum_desk', 'q_desk', 'prom_desk', 'freq_desk', 'sum_phone',
       'q_phone', 'prom_phone', 'freq_phone', 'sum_6a7', 'q_6a7', 'prom_6a7',
       'freq_6a7', 'sum_7a9', 'q_7a9', 'prom_7a9', 'freq_7a9', 'sum_9a13',
       'q_9a13', 'prom_9a13', 'freq_9a13', 'sum_13a16', 'q_13a16',
       'prom_13a16', 'freq_13a16', 'sum_16a18', 'q_16a18', 'prom_16a18',
       'freq_16a18', 'sum_18a21', 'q_18a21', 'prom_18a21', 'freq_18a21',
       'sum_21a06', 'q_21a06', 'prom_21a06', 'freq_21a06', 'sum_30ih',
       'q_30ih', 'prom_30ih', 'freq_30ih', 'fecha'],
      dtype='object')

In [26]:
#output13.to_csv("C:/Users/alopez/Desktop/indicadores/Indicadores_v3.csv", index = None, header=True, encoding='utf-8')

In [28]:
output13

,platform_id,federation_id,email,date_created,dias_registrado,condicion,sum_total,q_total,prom_total,freq_total,...,freq_18a21,sum_21a06,q_21a06,prom_21a06,freq_21a06,sum_30ih,q_30ih,prom_30ih,freq_30ih,fecha
0,62f57d0534b65e0009a00494,61af5331b7cda30009e76bbd,avatarignis@hotmail.com,2022-08-11,50,más de 30 días creado,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-30
1,62e7f19b1cf27900097fd759,61af5331b7cda30009e76bbd,cmenendezramos@outlook.com,2022-08-01,60,más de 30 días creado,7394.983333,18.0,410.832407,0.300000,...,0.033333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-30
2,62a1258f51cc5e000974fdfc,61af5331b7cda30009e76bbd,donbalon4@gmail.com,2022-06-08,114,más de 30 días creado,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-30
3,625ea607d39e810009d1c76f,61af5331b7cda30009e76bbd,eduplaza@gmail.com,2022-04-19,164,más de 30 días creado,271.000000,1.0,271.000000,0.006098,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-30
4,62eac2257328c30009b589e2,61af5331b7cda30009e76bbd,jure81@hotmail.com,2022-08-03,58,más de 30 días creado,9786.933333,40.0,244.673333,0.689655,...,0.155172,17.0,1.0,17.0,0.017241,785.9,7.0,112.283333,0.12069,2022-09-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49088,62e07855fd3be6000730a0a4,61af5331b7cda30009e76bbd,cubillosrosario@gmail.com,2022-07-26,66,más de 30 días creado,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-30
49089,62dfe39f916b390009ab32c0,61b2378168babc000837ddbb,cotigallo@gmail.com,2022-07-26,66,más de 30 días creado,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-30
49090,62e027687040e2000923e82d,61b2378168babc000837ddbb,taniaselaive@gmail.com,2022-07-26,66,más de 30 días creado,90.000000,2.0,45.000000,0.030303,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-30
49091,62e03a0b39ea4e0009883c4f,61b2378168babc000837ddbb,sromero@rdfmedia.cl,2022-07-26,66,más de 30 días creado,4713.000000,46.0,102.456522,0.696970,...,0.212121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-30


In [29]:
client = bigquery.Client()

table_id = "conexion-datos-rdf.indicadores_cia.indicadores_220930"

job_config = bigquery.LoadJobConfig(
        schema=[],
        write_disposition="WRITE_TRUNCATE",
        )
    
# Make an API request.
job = client.load_table_from_dataframe(output13, table_id, job_config=job_config) 

print(job.result())  # Wait for the job to complete.   

ArrowTypeError: Expected bytes, got a 'datetime.date' object